# Introduction

This notebooks show how to finish the setup and test the communication with a CSC.

## Setup

The first step, if you are planning on running this notebook is to copy it to your home folder. This will prevent multiple users from running the same notebook.

Next, before you can proceed to communicate with CSCs, make sure you loadded the correct notebook environment. In the upper right-hand side of the notebook there must be a `LSST` written before the empty circle, just bellow the notebook tabs. 

The next step is to make sure the opensplice setup file is loaded when the notebook server starts. To do that open a terminal, edit the file `~/notebooks/.user_setups` with your preferred text editor (both vi and emacs are available) and add the following:

```
source $OSPL_HOME/release.com
```

to the bottom of the file. 

Once you've done that you are ready to go. Restart the notebook kernel (go to Kernel > Restart kernel) and you are all set. 

Contact Tiago Ribeiro for help: `tribeiro` on slack or tribeiro@lsst.org. 

The library we use to communicat with CSCs is called salobj. You can find a developer guide for salobj at https://ts-salobj.lsst.io. A user guide will be made available soon. 

One of the main things to remember when using salobj is that the library is heavily designed on top of Python's `asyncio` standard module to provide concurrent capabilities. You will notice the presence of a lot of `await` on several ocasions which is the way you "wait" for a task to execute. 

We start by loading the library.

In [ ]:
from lsst.ts import salobj

The next step is to create a `Domain`. This class defined some basic communication infrastructure and can be shared with multiple resources.

In [ ]:
domain = salobj.Domain()

Now we finally get to point where we instantiate the class that handles the actual communication. In our system architecture world, the CSCs will listen for commands and output telemetry and events. To communicate with them we use a the `Remote`, which is a class that is capable of sending commands and receiving telemetry and events. 

The next cell starts a `Remote` for the `ATMCS`. If you know the name of the CSC you want to communicate with, you can replace the name in the cell bellow. The full list of available CSCs can be found here: https://ts-xml.lsst.io

In [ ]:
r = salobj.Remote(salobj.Domain(), "ATMCS")

This next cell is the first use of `asyncio`. The `start_task` is an `asyncio` task that starts up the communication. It is important to wait for this task to run or you may experience some timeouts when trying to communicate with the CSC. 

In [ ]:
await r.start_task

Once this is done you are ready to communicate with the CSC. The next cell will get a heartbeat from the CSC. This is the easiest way to check that the CSC is up and running and that the communication is working. 

In [ ]:
await r.evt_heartbeat.next(flush=True, timeout=5)

There are two ways one can perform state transitions on a CSC, you can either send the commands directly or use a helper method from salobj called `set_summary_state`. 

Let's exercise both. We start with `set_summary_state` to make sure the CSC is in `STANDBY`.

In [ ]:
await salobj.set_summary_state(r, salobj.State.STANDBY)

Note the output once you run the cell above. It shows the state transitions that the CSC when through.

Now that we know the CSC is in `STANDBY`, let's send a `start` command to take it to `DISABLE` and then a `enable` command to take it to `ENABLED`.


In [ ]:
await r.cmd_start.start()

In [ ]:
await r.cmd_enable.start()

A couple things to note. 

Both commands are followed by a `.start()` method. This is the method that actually returns the `asyncio` task for that command to be executed. It must follow all commands you are trying to execute. 

When we waited for the heartbeat there was an `evt_` before the name of the topic. When we send the command there was a `cmd_` this is how we differentiate between the three different types of topics. The one we haven't used yet is telemetry, which is preceded by `tel_`. 

In [ ]:
mount_azel_encoders = await r.tel_mount_AzEl_Encoders.next(flush=True, timeout=5)

In [ ]:
print(mount_azel_encoders) # This will print a really long ouput.

You can check what are all the available commands, events and telemetry for the CSC you are trying to communicate with by doing the following.

In [ ]:
print(r.salinfo.command_names)

In [ ]:
print(r.salinfo.event_names)

In [ ]:
print(r.salinfo.telemetry_names)